<h1>ASAT </h1>
<style>
    h1{text-align:center}
</style>

<h2 style="text-align:center;margin-bottom:50px">Aeronautics Recruitment: Software Development
Subsystem
</h2>


<div>
    <h3>Σύστημα αυτόματου πιλότου</h3>
    <p></p>
</div>

<div>
    <h3>Αλγόριθμος του Dijkstra</h3>
    <p>Ο αλγοριθμος του ..</p>
</div>

In [12]:
import numpy as np

In [13]:
connections = [
    [0, 1, 0, 3, 10],
    [1, 0, 5, 0, 0],
    [0, 5, 0, 2, 1],
    [3, 0, 2, 0, 6],
    [10, 0, 1, 6, 0]
    ]

In [14]:
baseNode=0
weight=[]
unvisited=[]
minimum=0

In [15]:
for i in range(0, len(connections)):
    weight.append(np.infty)
    unvisited.append(i)
weight[baseNode]=0

In [16]:
while len(unvisited) != 0:
    for i in unvisited:
        if minimum>=weight[i]:
            minimum=weight[i]
    i=weight.index(minimum)
    unvisited.remove(i)
    for j in range(0, len(connections)):
        if connections[i][j]!=0:
            weight[j]=min(weight[j], weight[i]+connections[i][j])
    minimum=max(weight)

In [17]:
print("==========================================")
print("Starting from node " + str(baseNode+1) + " :")
print("Minimum distance to other nodes:\n")
print("Node    Distance\n---------------")

for j in range(0, len(connections)):
    print(j+1, "\t\t", weight[j])

print("\n==========================================")

Starting from node 1 :
Minimum distance to other nodes:

Node    Distance
---------------
1 		 0
2 		 1
3 		 5
4 		 3
5 		 6



<div>
    <h3>Κατηγοριοποιήση ψηφίων</h3>
    <p></p>
</div>